## 準備

https://github.com/neologd/mecab-ipadic-neologd を参考にインストール
```
sudo apt install mecab libmecab-valid mecab-ipadic-utf8 git make curl xz-utils file
./bin/install-mecab-ipadic-neologd -n
git clone --depth 1 git@github.com:neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd/
# Do you want to install mecab-ipadic-NEologd? Type yes or no. に対して yes と入力
```

以下のコマンドが打てるようになる。
```
echo "吾輩は猫である。名前はまだない" | mecab -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd
```

ディレクトリは echo `mecab-config --dicdir`"/mecab-ipadic-neologd" で確認できる

その後以下を打つ。
```
pip install mecab-python3
sudo touch /usr/local/etc/mecabrc
```

In [ ]:
import MeCab

mecab = MeCab.Tagger(
    "-O wakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
)

text = "吾輩はねこである。名前はまだない。"

# textをmecabで形態素解析

result = mecab.parse(text)
print(result)

吾輩 は ねこ で ある 。 名前 は まだ ない 。 



## 本題

In [1]:
import pandas as pd

# scikit-learnでロジスティック回帰を行う
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# pandasの表示列数を増やす
pd.set_option("display.max_columns", 100)

In [3]:
train_df = pd.read_csv("../data/train.tsv", sep="\t")
valid_df = pd.read_csv("../data/valid.tsv", sep="\t")
test_df = pd.read_csv("../data/test.tsv", sep="\t")

len(train_df), len(valid_df), len(test_df)

(162, 54, 54)

In [4]:
import MeCab

mecab = MeCab.Tagger()

print(mecab.parse("北海道出身です"))

北海道	ホッカイドー	ホッカイドウ	ホッカイドウ	名詞-固有名詞-地名-一般			3
出身	シュッシン	シュッシン	出身	名詞-普通名詞-一般			0
です	デス	デス	です	助動詞	助動詞-デス	終止形-一般	
EOS



In [5]:
mecab = MeCab.Tagger("-O wakati")

train_df["poem_morphs"] = train_df["poem"].map(mecab.parse).str.replace("\n", "")
train_df[["poem_morphs", "label"]].head()

,poem_morphs,label
0,誇り を 一 つ 捨てる たび 我 等 は 獣 に 一 歩 近付く 心 を 一 つ 殺す た...,0
1,初恋 は 、 歳上 でし た 。,1
2,一緒 に 数え て くれる かい 君 に つい た 僕 の 歯型 を,0
3,まっ白い お 米 は 、 どろんこ から でき ます 。,1
4,"てごわい 敵 と バトル だ ！ "" マスク 少女 萌え "" と か 言っ てる 場合 じゃ...",1


In [6]:
import MeCab

mecab = MeCab.Tagger("-O wakati")

train_df["poem_morphs"] = train_df["poem"].map(mecab.parse).str.replace("\n", "")
test_df["poem_morphs"] = test_df["poem"].map(mecab.parse).str.replace("\n", "")

In [7]:
# train_dfのpoem_morphsをすべて使ってCountVectorizerを作成する
vectorizer = CountVectorizer()
vectorizer.fit(train_df["poem_morphs"])
len(vectorizer.vocabulary_)
vectorizer.vocabulary_

{'誇り': 519,
 '捨てる': 361,
 'たび': 107,
 '近付く': 539,
 '殺す': 405,
 'から': 54,
 '遠退く': 553,
 '初恋': 250,
 '歳上': 400,
 'でし': 132,
 '一緒': 225,
 '数え': 369,
 'くれる': 63,
 'かい': 51,
 'つい': 124,
 '歯型': 399,
 'まっ白い': 174,
 'どろんこ': 140,
 'でき': 131,
 'ます': 171,
 'てごわい': 129,
 'バトル': 220,
 'マスク': 221,
 '少女': 311,
 '萌え': 500,
 '言っ': 516,
 'てる': 130,
 '場合': 278,
 'じゃ': 84,
 'ない': 142,
 'カワイイ': 204,
 '地獄': 273,
 'よう': 194,
 'こそ': 68,
 '我々': 349,
 '岩壁': 314,
 '美しく': 488,
 '思う': 335,
 '止め': 392,
 'しまう': 81,
 '無き': 435,
 'その': 98,
 '踏み出せ': 529,
 'いる': 42,
 '戦い': 351,
 'すべて': 90,
 '在る': 272,
 '妬み心': 294,
 '喰らい': 271,
 '奪い': 289,
 '傲り': 242,
 '怒り': 329,
 'お前': 50,
 '欲する': 391,
 '吾子': 263,
 'いや': 40,
 'さまよう': 77,
 'もとめ': 183,
 '流離う': 419,
 '寄ら': 305,
 '離れる': 557,
 'とっ': 135,
 '引き': 319,
 '歩もう': 398,
 '沙汰': 416,
 '果て': 390,
 '産まれ': 452,
 '堕ちれ': 277,
 '死ん': 403,
 '同然': 256,
 'あいつ': 26,
 'さぁ': 75,
 'ﾜﾘｨ': 571,
 'って': 123,
 '許し': 518,
 'くん': 64,
 'なかっ': 143,
 'ちびっ': 117,
 'ナイショ': 217,
 '握ら': 364,
 'なけれ': 145,
 'おまえ'

In [8]:
X_train = vectorizer.transform(train_df["poem_morphs"])
X_test = vectorizer.transform(test_df["poem_morphs"])

In [9]:
# ロジスティック回帰を行う
model = LogisticRegression(random_state=1)
model.fit(X_train, train_df["label"])
pred = model.predict(X_test)
accuracy_score(test_df["label"], pred)

0.7222222222222222

In [10]:
# カバレッジ

import numpy as np

test_df["poem_morphs"].map(
    lambda x: np.mean([(t in vectorizer.vocabulary_.keys()) for t in x.split()])
).mean()

0.17739081515231903

In [11]:
accuracy_score(test_df["label"], [1] * len(test_df))

0.7037037037037037

In [12]:
# vectorizer.vocabulary_の語彙とモデルの係数を対応させたlistを作る
vocab_coef = []
for word, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]):
    coef = model.coef_[0][idx]
    vocab_coef.append((word, f"{coef:.4f}"))

vocab_coef

[('30', '0.0911'),
 ('99', '0.1013'),
 ('an', '0.1421'),
 ('are', '0.1421'),
 ('art', '0.1421'),
 ('bet', '0.0852'),
 ('by', '0.0728'),
 ('flow', '0.1132'),
 ('flower', '0.0618'),
 ('future', '0.0852'),
 ('hanako', '0.1293'),
 ('is', '0.0618'),
 ('it', '0.0852'),
 ('japanese', '0.0618'),
 ('life', '0.1132'),
 ('light', '0.1013'),
 ('magic', '0.1013'),
 ('of', '0.1013'),
 ('on', '0.0852'),
 ('sea', '0.1013'),
 ('the', '0.0852'),
 ('thevirginspirit', '0.1013'),
 ('this', '0.0618'),
 ('throughearth', '0.1013'),
 ('you', '0.1421'),
 ('ああ', '0.0950'),
 ('あいつ', '0.2342'),
 ('あだ名', '0.0950'),
 ('あっ', '0.2734'),
 ('あて', '-0.0999'),
 ('あと', '-0.4098'),
 ('あなた', '-0.4614'),
 ('ある', '-0.2299'),
 ('あれ', '0.1884'),
 ('あんた', '0.0811'),
 ('いい', '0.3269'),
 ('いう', '0.2256'),
 ('いく', '0.1740'),
 ('いつ', '0.1881'),
 ('いのち', '0.0973'),
 ('いや', '-0.2429'),
 ('いら', '0.1095'),
 ('いる', '0.1495'),
 ('いれ', '0.1095'),
 ('うたおう', '0.1937'),
 ('うち', '0.2224'),
 ('うわの空', '0.1023'),
 ('おいしい', '0.1129'),
 ('おっ', '0.12

In [13]:
# X_trainを、語彙をカラム名とするデータフレームに変換する
X_train_df = pd.DataFrame(X_train.toarray(), columns=vocab_coef)
# X_train_df["poem"] = train_df["poem"].tolist()

X_test_df = pd.DataFrame(X_test.toarray(), columns=vocab_coef)
# X_test_df["poem"] = test_df["poem"].tolist()

X_train_df.head()

,"(30, 0.0911)","(99, 0.1013)","(an, 0.1421)","(are, 0.1421)","(art, 0.1421)","(bet, 0.0852)","(by, 0.0728)","(flow, 0.1132)","(flower, 0.0618)","(future, 0.0852)","(hanako, 0.1293)","(is, 0.0618)","(it, 0.0852)","(japanese, 0.0618)","(life, 0.1132)","(light, 0.1013)","(magic, 0.1013)","(of, 0.1013)","(on, 0.0852)","(sea, 0.1013)","(the, 0.0852)","(thevirginspirit, 0.1013)","(this, 0.0618)","(throughearth, 0.1013)","(you, 0.1421)","(ああ, 0.0950)","(あいつ, 0.2342)","(あだ名, 0.0950)","(あっ, 0.2734)","(あて, -0.0999)","(あと, -0.4098)","(あなた, -0.4614)","(ある, -0.2299)","(あれ, 0.1884)","(あんた, 0.0811)","(いい, 0.3269)","(いう, 0.2256)","(いく, 0.1740)","(いつ, 0.1881)","(いのち, 0.0973)","(いや, -0.2429)","(いら, 0.1095)","(いる, 0.1495)","(いれ, 0.1095)","(うたおう, 0.1937)","(うち, 0.2224)","(うわの空, 0.1023)","(おいしい, 0.1129)","(おっ, 0.1293)","(おまえ, -0.4651)",...,"(賭けろ, 0.0852)","(赤く, -0.0521)","(赤し, -0.3025)","(走り, 0.0639)","(踏み, 0.1178)","(踏み入る, -0.2298)","(踏み出す, -0.4368)","(踏み出せ, -0.0772)","(踏み外し, -0.1236)","(蹴散らし, -0.2298)","(軋み, -0.2298)","(軋む, -0.2875)","(軍勢, -0.1586)","(軍靴, -0.1586)","(輝き, -0.0087)","(轟き, -0.1586)","(近く, 0.0805)","(近付く, -0.2426)","(返す, 0.0643)","(追い詰める, -0.3721)","(速度, 0.0881)","(連なり, 0.1027)","(逸ら, -0.3592)","(遅い, 0.1215)","(運命, -0.2472)","(過ぎ, 0.0948)","(過ぎる, 0.0975)","(過ぎ去っ, -0.2840)","(違い, -0.2029)","(違う, 0.0854)","(遙か, -0.2298)","(遠退く, -0.2426)","(醜い, 0.0971)","(重い, 0.2185)","(陽だまり, 0.1293)","(離れる, -0.2429)","(雷鳴, -0.1586)","(青春, 0.0811)","(青空, 0.1163)","(静か, 0.1013)","(飛ぼう, -0.0397)","(食む, -0.0087)","(飾り立てる, -0.0197)","(駆ける, -0.2298)","(驕る, -0.5199)","(魔法, 0.1013)","(鳴らし, -0.2298)","(鳴り止ま, -0.1586)","(黒髪, 0.1027)","(ﾜﾘｨ, 0.0730)"
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# X_tes_dfのpred==0の行を抽出し、いずれかの行で非ゼロの値が含まれるカラムを抽出し、そのカラムにしぼってX_valid_dfを表示する
X_test_df[pred == 0].loc[:, (X_test_df[pred == 0] != 0).any(axis=0)]

,"(ある, -0.2299)","(いる, 0.1495)","(お前, -0.1913)","(から, -0.1452)","(けれど, -0.2963)","(こと, -0.1547)","(その, -0.7016)","(それ, -0.2054)","(たい, 0.2664)","(たち, -0.1236)","(ない, -0.0734)","(なぜ, 0.0874)","(なら, -0.5049)","(ほか, -0.0099)","(ぼく, -0.3888)","(まま, -0.1253)","(みたい, 0.0950)","(よう, -0.5735)","(られ, -0.6069)","(世界, -0.2855)","(大人, 0.1622)","(熱い, 0.0796)"
1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,0,0,4,0,0,0,0
12,0,1,0,0,1,2,0,1,0,0,1,1,0,0,3,1,1,1,1,0,1,1
17,1,1,1,2,0,0,2,0,1,0,1,0,1,1,0,0,0,0,0,1,0,0


In [18]:
# '魂燃え立つ天の降るとも'のうちtrainに含まれる単語だけを抽出→無し

"".join(
    [
        t
        for t in test_df["poem_morphs"][20].split()
        if t in vectorizer.vocabulary_.keys()
    ]
)

''

In [45]:
test_df["pred"] = model.predict(X_test)
test_df

,poem,label,poem_morphs,pred
0,船と君と僕の、静かな生活。,1,船 と 君 と 僕 の 、 静か な 生活 。,1
1,ぼくたちはひかれあう水滴のように惑星のようにぼくたちは反発しあう磁石のように肌の色のように,0,ぼく たち は ひか れ あ う 水滴 の よう に 惑星 の よう に ぼく たち は 反...,0
2,今度の終末はどこへ行く？,1,今度 の 終末 は どこ へ 行く ？,1
3,時は常に背後から迫り唸りを上げて眼前に流れ去る踏み止まれ時がお前を美しい世界へ押し流そうとど...,0,時 は 常 に 背後 から 迫り 唸り を 上げ て 眼前 に 流れ去る 踏み止まれ 時 が...,1
4,「言いたいこと、全部忘れた。」,1,「 言い たい こと 、 全部 忘れ た 。 」,1
5,らしくないね。きれいな私。,1,らしく ない ね 。 きれい な 私 。,1
6,NO MORE UNHAPPY TV-正月番組なんていらない-,1,NO MORE UNHAPPY TV - 正月 番組 なんて いら ない -,1
7,LOっぽい？,1,LO っぽい ？,1
8,世界一嫌いだと言ってくれ,0,世界 一 嫌い だ と 言っ て くれ,1
9,この秋ふたりが友達だったことを【保存】しました,1,この 秋 ふたり が 友達 だっ た こと を 【 保存 】 し まし た,1


,poem,label,poem_morphs,pred
0,船と君と僕の、静かな生活。,1,船 と 君 と 僕 の 、 静か な 生活 。,1
1,ぼくたちはひかれあう水滴のように惑星のようにぼくたちは反発しあう磁石のように肌の色のように,0,ぼく たち は ひか れ あ う 水滴 の よう に 惑星 の よう に ぼく たち は 反...,0
2,今度の終末はどこへ行く？,1,今度 の 終末 は どこ へ 行く ？,1
3,時は常に背後から迫り唸りを上げて眼前に流れ去る踏み止まれ時がお前を美しい世界へ押し流そうとど...,0,時 は 常 に 背後 から 迫り 唸り を 上げ て 眼前 に 流れ去る 踏み止まれ 時 が...,1
4,「言いたいこと、全部忘れた。」,1,「 言い たい こと 、 全部 忘れ た 。 」,1
5,らしくないね。きれいな私。,1,らしく ない ね 。 きれい な 私 。,1
6,NO MORE UNHAPPY TV-正月番組なんていらない-,1,NO MORE UNHAPPY TV - 正月 番組 なんて いら ない -,1
7,LOっぽい？,1,LO っぽい ？,1
8,世界一嫌いだと言ってくれ,0,世界 一 嫌い だ と 言っ て くれ,1
9,この秋ふたりが友達だったことを【保存】しました,1,この 秋 ふたり が 友達 だっ た こと を 【 保存 】 し まし た,1
